# 01 – Data Overview (PySpark)

This notebook introduces the Taobao CTR dataset. We will load up to **500k rows** from each CSV file using PySpark (to handle large data volumes) and inspect the schema and basic statistics. 

**Note on Kaggle download**: If you have Kaggle API credentials set up, you can automatically download the dataset. Uncomment the lines in the code cell below and ensure your `kaggle.json` credentials file is in the correct location (`~/.kaggle`). Otherwise, place the CSV files manually into `data/raw`. 

* Data sources:
  * `raw_sample.csv` – user impressions and clicks
  * `ad_feature.csv` – advertisement metadata
  * `user_profile.csv` – user demographics
  * `behavior_log.csv` – user behaviour logs (page view, cart, favourite, purchase)


In [ ]:
from pyspark.sql import SparkSession
import os

# Start Spark session
spark = (
    SparkSession.builder
        .appName("CTR_Data_Overview")
        .config("spark.sql.shuffle.partitions", 200) 
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN") # Reduce log verbosity
print("Spark version:", spark.version)

# Define project and data paths
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

raw_dir = os.path.join(project_root, "data", "raw")
print("Raw data directory:", raw_dir)

# Expected file names
file_names = {
    "raw_sample": "raw_sample.csv",
    "ad_feature": "ad_feature.csv",
    "user_profile": "user_profile.csv",
    "behavior_log": "behavior_log.csv",
}

# Verify that files exist
missing = []
for key, fname in file_names.items():
    

_IncompleteInputError: incomplete input (368411133.py, line 31)